Installing Dependencies...

mamba create -y -n geodock python=3.10 ; mamba activate geodock; mamba install -y -c conda-forge ipykernel && \
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 && \
mamba install -y -c conda-forge pdbfixer && \
pip install scipy

<!-- mamba install -y pytorch==1.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia && \
mamba install -y -c conda-forge openmm swig pdbfixer openbabel smina fpocket vina && \
mamba install -y -c bioconda pyrsistent foldseek &&\
mamba install -y -c pyg pyg=2.3.1=py310_torch_1.13.0_cu117 pytorch-cluster=1.6.1=py310_torch_1.13.0_cu117 pytorch-sparse=0.6.17=py310_torch_1.13.0_cu117 pytorch-scatter=2.1.1=py310_torch_1.13.0_cu117 && \
mamba install -y -c bioconda -c "dglteam/label/cu117" dgl -->

In [11]:
import os
import sys
import torch
# os.environ['TORCH_HOME'] = '/central/groups/MazmanianLab/joeB/cache/GeoDock/geodock/weights'
os.environ['TORCH_HOME'] = '/mnt/nvme0/jbok/cache'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:32'

In [ ]:
# Installing pytorch for CUDA 11.7

pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install geodock
!pip install torch-geometric
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install py3Dmol

# download model weights
# !cd /central/groups/MazmanianLab/joeB/docking && git clone https://github.com/Graylab/GeoDock.git
# !cd /mnt/nvme0/jbok/docking && git clone https://github.com/Graylab/GeoDock.git

In [1]:
import os
import sys
import torch
# os.environ['TORCH_HOME'] = '/central/groups/MazmanianLab/joeB/cache/GeoDock/geodock/weights'
os.environ['TORCH_HOME'] = '/mnt/nvme0/jbok/cache'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [2]:
from geodock.GeoDockRunner import GeoDockRunner
torch.cuda.empty_cache()
# ckpt_file = "/central/groups/MazmanianLab/joeB/docking/GeoDock/geodock/weights/dips_0.3.ckpt"
ckpt_file = "/mnt/nvme0/jbok/docking/GeoDock/geodock/weights/dips_0.3.ckpt"
geodock = GeoDockRunner(ckpt_file=ckpt_file)

In [3]:
# # input_path = "/central/groups/MazmanianLab/joeB/Microbiota-Immunomodulation/Microbiota-Immunomodulation/data/interim/scratch"
# input_path = "/mnt/nvme0/jbok/Microbiota-Immunomodulation/data/scratch"
# pred = geodock.dock(
#     partner1=os.path.join(input_path, "CD28_AF-P10747-F1-model_v4.pdb"),
#     partner2=os.path.join(input_path, "CD80_AF-P33681-F1-model_v4.pdb"),
#     out_name="CD28-CD80_GeoDock_Refined_r2",
#     do_refine=True,
#     use_openmm=True
# )

Docking tR01 - POC monomers in all pairwise matches

In [4]:
import pandas as pd
# import concurrent.futures
wkdir = '/mnt/nvme0/jbok/Microbiota-Immunomodulation'
pdb_paths = wkdir + '/data/scratch/ranked_0_pdbs'
geodock_outdir = wkdir + '/data/scratch/geodock_results'


In [5]:
# Function to construct the new path
def construct_new_path(old_path, new_folder, index):
    # Extract the filename part (like 'P35590') from the old path
    filename_part = old_path.split('/')[index].split('_')[0]
    # Construct the new path with the new folder and modified filename
    new_path = os.path.join(new_folder, f'{filename_part}_ranked_0.pdb')
    return new_path

# Read the CSV file

df = pd.read_csv(f"{wkdir}/data/scratch/docking-pairs_rank0-pdbs.csv")
# df.head
# # Update the paths
df['path1'] = df['path1'].apply(lambda x: construct_new_path(x, pdb_paths, -2))
df['path2'] = df['path2'].apply(lambda x: construct_new_path(x, pdb_paths, -2))
df['outpath'] = df['outpath'].apply(lambda x: construct_new_path(x, geodock_outdir, -1))
df['outpath'] = df['outpath'].str.replace('.pdb', '', regex=False)

# df
dock_df = df.head(5)


In [6]:
torch.cuda.set_per_process_memory_fraction(1, 0)
# torch.cuda.set_per_device_memory_fraction(0.8, 0)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

TypeError: Invalid type for fraction argument, must be `float`

In [7]:
geodock.dock(
    partner1=dock_df['path1'][1],
    partner2=dock_df['path2'][1],
    out_name=dock_df['outpath'][1],
    do_refine=False,
    use_openmm=False,
)

Completed embedding in 3.21 seconds.


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.66 GiB (GPU 0; 79.35 GiB total capacity; 74.93 GiB already allocated; 3.41 GiB free; 75.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
# def dock(path1, path2, outpath):
#     # Extract the filename for out_name from outpath
#     out_name = os.path.basename(outpath)

#     # Call the geodock.dock function with the provided arguments
#     torch.cuda.empty_cache()
#     geodock.dock(
#         partner1=path1,
#         partner2=path2,
#         out_name=out_name,
#         do_refine=True,
#         use_openmm=True
#     )

In [7]:
# dock_df = df.head(10)
# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#     # Submitting tasks to the executor
#     futures = [executor.submit(dock, row['path1'], row['path2'], row['outpath']) for index, row in dock_df.iterrows()]
#     # Waiting for all tasks to complete (optional, depending on your use case)
#     concurrent.futures.wait(futures)

In [8]:
# torch.cuda.empty_cache()
dock_df = df.head(5)
for index, row in dock_df.iterrows():
    outpath = row['outpath']
    print(f"Processing index: {index}")
    # Check if the file exists
    if os.path.exists(outpath):
        print(f"File {outpath} already exists. Skipping this iteration.")
        continue
    try:
        geodock.dock(
            partner1=row['path1'],
            partner2=row['path2'],
            out_name=row['outpath'],
            do_refine=False,
            use_openmm=False,
        )
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print(f"GPU Out of Memory error at index {index}. Skipping this iteration.")
        else:
            raise  # Re-raise the exception if it's not an OOM error
    except Exception as e:
        print(f"An error occurred at index {index}: {e}")

Processing index: 0
Completed embedding in 2.51 seconds.
GPU Out of Memory error at index 0. Skipping this iteration.
Processing index: 1
Completed embedding in 2.44 seconds.
GPU Out of Memory error at index 1. Skipping this iteration.
Processing index: 2
Completed embedding in 2.18 seconds.
File '/mnt/nvme0/jbok/Microbiota-Immunomodulation/data/scratch/geodock_results/P35590_ranked_0.pdb' deleted successfully.
Completed docking in 2.09 seconds.
Processing index: 3
Completed embedding in 2.17 seconds.
File '/mnt/nvme0/jbok/Microbiota-Immunomodulation/data/scratch/geodock_results/P35590_ranked_0.pdb' deleted successfully.
Completed docking in 2.17 seconds.
Processing index: 4
Completed embedding in 2.63 seconds.
GPU Out of Memory error at index 4. Skipping this iteration.


In [ ]:
# !nvidia-smi
# srun --mem=40G --time=01:00:00 --gres=gpu:1 --pty bash -i

# if re.search('geodock/lib', os.environ['LD_LIBRARY_PATH']) == None:
#     os.environ['LD_LIBRARY_PATH'] = os.environ['CONDA_PREFIX'] + "/lib:" + os.environ['LD_LIBRARY_PATH']

# import openmm as mm

In [ ]:
if re.search('geodock/lib', os.environ['LD_LIBRARY_PATH']) == None:
    os.environ['LD_LIBRARY_PATH'] = os.environ['CONDA_PREFIX'] + "/lib:" + os.environ['LD_LIBRARY_PATH']

In [ ]:
import os
import sys
import torch
import re

os.environ['TORCH_HOME'] = '/central/groups/MazmanianLab/joeB/cache/GeoDock/geodock/weights'
from geodock.GeoDockRunner import GeoDockRunner
torch.cuda.empty_cache()
ckpt_file = "/central/groups/MazmanianLab/joeB/docking/GeoDock/geodock/weights/dips_0.3.ckpt"
geodock = GeoDockRunner(ckpt_file=ckpt_file)

input_path = "/central/groups/MazmanianLab/joeB/Microbiota-Immunomodulation/Microbiota-Immunomodulation/data/interim/scratch"
pred = geodock.dock(
    partner1=os.path.join(input_path, "CD28_AF-P10747-F1-model_v4.pdb"),
    partner2=os.path.join(input_path, "CD80_AF-P33681-F1-model_v4.pdb"),
    out_name="CD28-CD80_GeoDock",
    do_refine=True,
    use_openmm=True,
)

In [ ]:
input_path = "/central/groups/MazmanianLab/joeB/Microbiota-Immunomodulation/Microbiota-Immunomodulation/data/interim/scratch"
pred = geodock.dock(
    partner1=os.path.join(input_path, "PD1_AF-Q15116-F1-model_v4.pdb"),
    partner2=os.path.join(input_path, "PDL1_AF-Q9NZQ7-F1-model_v4.pdb"),
    out_name="PD1-PDL1_GeoDock",
    do_refine=False,
    use_openmm=False,
)

In [ ]:
# !export LD_LIBRARY_PATH=/home/.conda/envs/immune-builder/lib
# !export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
# !echo $LD_LIBRARY_PATH

In [ ]:
# from openmm.app import *
from openmm import *
# from openmm.unit import *
# from sys import stdout


In [ ]:
import openmm as mm

In [ ]:
# !mamba install -y -c conda-forge openmm cudatoolkit=11.7
# ln -sf /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /home/jbok/miniforge3/envs/geodock/lib/libstdc++.so.6
# /home/anavani/anaconda3/envs/dmcgb/bin/../lib/libstdc++.so.6

